In [1]:
##############################################################################
#The app works around 4 major steps
#    (1) Configure the user input
#    (2) Confirm the user input + submit the job
#    (3) Monitor the process during the run
#    (4) Display the final results
#
#These steps are linked via the following 
#    (1) --> (2): user_inputs, a dictionary of all the user inputs
#    (1) --> (2): mpuc3_code, a code for marketplace usercase 3
#    (2) --> (3): process, the process (MPusercase3 CalcJob) submitted to AiiDA 
#    (3) --> (4): output, the ArrayData output from the CalcJob
#
#This is an early prototype that is 'stapled' from the pizza example in aiidalab_widgets_base
#as well as QE example from aiidalab_qe
#
#Specific notes/issues on the steps:
#    Step 1:
#        * Perhaps there should be a step 0 that allows the user to choose a model,
#          then the widget can be dynamically updated
#        * The code setup is rather ugly, and creating a custom code crashes
#        * The user should be alerted that if they try to hit submit without 
#          choosing a code, that they need to choose a code first
#    Step 2:
#    Step 3:
#        * This is taken almost directly from QE, and is the part I understand the least
#        * Clicking on most outputs causes a crash
#        *** I can't seem to set the 'output' propery correctly
#    Step 4:
#        *** Because I can't set the output property in step3 correctly, this step displays nothing
#        * The current display is very ugly and taken directly from Step2, and probably be changed to
#         something nicer
##############################################################################

In [1]:
import logging

from uc3.logger import OutputWidgetHandler

logger = logging.getLogger("aiidalab_mp_uc3")
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Remove/Add comment of the next line to show/hide logs.
# handler.show_logs()

handler.clear_logs()
logger.info('Starting program')

In [2]:
import ipywidgets as ipw

from aiidalab_widgets_base import WizardAppWidget

from uc3.steps import (
    ComputerCodeSetupStep,
    ConfigureUserInputStep,
    ConfirmUserInputStep,
    MonitorProcessStep,
    DisplayFinalOutput,
)

computer_code_setup_step = ComputerCodeSetupStep(auto_advance=True)
configure_userinput_step = ConfigureUserInputStep(auto_advance=True, description_label_default=[  ['dmf', 'dmf', 0],
  ['gfr', 'gfr', 0],
  ['ch4mf', 'ch4mf', 0],
  ['h2omf', 'h2omf', 0],
  ['surfa', 'surfa', 0],
  ['amf', 'amf', 0],
  ['macropor', 'macropor', 0],
  ['macrotor', 'macrotor', 0],
  ['k1', 'k1', 0],
  ['ea1', 'ea1', 0],
  ['k2', 'k2', 0],
  ['ea2', 'ea2', 0],
  ['koeq', 'koeq', 0],
  ['dhh20', 'dhh20', 0]
])


confirm_userinput_step = ConfirmUserInputStep(auto_advance=True)
monitorprocess_step = MonitorProcessStep(auto_advance=True)
displayfinaloutput_step = DisplayFinalOutput(auto_advance=False)

ipw.dlink(
    (configure_userinput_step, "user_inputs"),
    (confirm_userinput_step, "user_inputs"),
)
ipw.dlink(
    (computer_code_setup_step, "mpuc3_code"),
    (confirm_userinput_step, "mpuc3_code"),
)
ipw.dlink(
    (confirm_userinput_step, "process"),
    (monitorprocess_step, "process"),
    transform=lambda x: x.uuid if x is not None else None
)
ipw.dlink(
    (monitorprocess_step, "output"),
    (displayfinaloutput_step, "output"),
)

# Setup the app by adding the various steps in order.
WizardAppWidget(
    steps=[
        ("Setup Computer & Code", computer_code_setup_step),
        ("Configure User Input", configure_userinput_step),
        ("Confirm User Input", confirm_userinput_step),
        ("Monitor Process", monitorprocess_step),
        ("Display Results", displayfinaloutput_step),
    ]
)

WizardAppWidget(children=(HBox(children=(Button(description='Previous step', disabled=True, icon='step-backwar…

In [1]:
#display(monitorprocess_step)
#monitorprocess_step.process = "8cc8cbe9-81ab-4821-8dad-b25e73de9327"
#monitorprocess_step.output
#monitorprocess_step.state
#monitorprocess_step.state
#display(widget)

In [21]:
import json
import uuid
import logging
import subprocess
from copy import copy
from pathlib import Path

import ipywidgets as ipw
import shortuuid
import traitlets
from aiida.engine import ProcessState, submit
from aiida.orm import ArrayData, Code, Dict, Str, ProcessNode, load_code, load_node
from aiida.plugins import CalculationFactory
from aiidalab_widgets_base import (
    AiidaNodeViewWidget,
    ProcessMonitor,
    ProcessNodesTreeWidget,
    WizardAppWidgetStep,
)

from uc3.base_widgets_mod import ComputationalResourcesWidget
import matplotlib.pyplot as plt
from io import BytesIO
import base64

from ipywidgets import AppLayout, Button, Layout, Label
import numpy as np
from numpy import random
from scipy import ndimage
from traitlets import HasTraits, Instance, observe

class TestOutput(HasTraits):
    area_flux = ndimage.gaussian_filter(random.normal(50, 5, size=(512, 512)), sigma=10).ravel()
    volume_flux = ndimage.gaussian_filter(random.normal(100, 10, size=(512, 512)), sigma=10).ravel()
    particle_size = ndimage.gaussian_filter(random.normal(10, 2, size=(512, 512)), sigma=10).ravel()
    
    def get_array(self, foo):
        return [1,2,3,4,5]

class DisplayFinalOutput(ipw.VBox, WizardAppWidgetStep):

#     output = traitlets.Instance(ArrayData, allow_none=True)
    output = traitlets.Instance(TestOutput, allow_none=True)

    def __init__(self, **kwargs):
        self.output_figure = plt.figure()
        self.final_output = ipw.Output()

        super().__init__([self.final_output], **kwargs)

    @traitlets.observe("output")
    def _call_observe(self, change):
        self._observe_configuration(self, change)

    def _observe_configuration(self, _, change):
        "Format and show the user inputs."
        if change["new"]:
            output = self.output
            area_flux = output.get_array("area_flux")
            volume_flux = output.get_array("volume_flux")
            particle_size = output.get_array("particle_size")
            display_dict = {
                "Area Flux": area_flux,
                "Volume Flux": volume_flux,
                "Particle Size": particle_size,
            }

            # Create a plot of the data
            x = [1, 2, 3, 4, 5]
            y = [2, 4, 5, 6, 7]
            plt.plot(x, y)
            plt.xlabel('X axis')
            plt.ylabel('Y axis')

            # Save the plot to a buffer
            buffer = BytesIO()
            plt.savefig(buffer, format='svg')
            plt.close()

            # Convert the buffer to a base64-encoded string and create the SVG image
            svg_data = buffer.getvalue().decode()
            base64_data = base64.b64encode(svg_data.encode()).decode()
            svg_image = f'<img src="data:image/svg+xml;base64,{base64_data}">'

            # Show the user inputs and the plot
            self.final_output.clear_output(wait=True)
            with self.final_output:
#                 display(ipw.HTML(f"<h4>Configuration</h4><pre>{json.dumps(display_dict, indent=2)}</pre>"))
                display(ipw.HTML(svg_image))

            self.state = self.State.SUCCESS
        else:
            self.final_output.value = (
                "<h4>Configuration</h4>[Please configure user inputs]"
            )


output_data = TestOutput()

output_widget = DisplayFinalOutput(output=output_data)

AppLayout(center=output_widget)

AppLayout(children=(DisplayFinalOutput(children=(Output(),), layout=Layout(grid_area='center')),), layout=Layo…